In [1]:
# Install necessary libraries
!pip install litellm flask pandas scikit-learn pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 14.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 40.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 KB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 MB 29.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 KB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.6 MB/s eta 0:00:000:00:01


In [2]:
import os

# Create directories for your project
os.makedirs('litellm_ml_app/app/static', exist_ok=True)
os.makedirs('litellm_ml_app/app/templates', exist_ok=True)

# Create necessary Python files
open('litellm_ml_app/app/__init__.py', 'w').close()
open('litellm_ml_app/app/routes.py', 'w').close()
open('litellm_ml_app/app/model.py', 'w').close()
open('litellm_ml_app/app/utils.py', 'w').close()

print("App structure created successfully.")


App structure created successfully.


In [3]:
model_code = '''
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import litellm

def load_data(file_path):
    data = pd.read_csv(file_path)
    return data

def preprocess_data(data):
    # Implement your preprocessing steps here
    X = data.drop('target', axis=1)
    y = data['target']
    return X, y

def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = litellm.LLModel()  # Initialize the LiteLLM model
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    return model, accuracy

def save_model(model, file_path):
    model.save(file_path)
'''

with open('litellm_ml_app/app/model.py', 'w') as f:
    f.write(model_code)

print("Model code created successfully.")


Model code created successfully.


In [4]:
routes_code = '''
from flask import Flask, request, render_template, redirect, url_for
import os
from .model import load_data, preprocess_data, train_model, save_model

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads'

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/train', methods=['POST'])
def train():
    if 'datafile' not in request.files:
        return redirect(url_for('home'))
    
    file = request.files['datafile']
    if file.filename == '':
        return redirect(url_for('home'))
    
    if file:
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        file.save(file_path)
        data = load_data(file_path)
        X, y = preprocess_data(data)
        model, accuracy = train_model(X, y)
        save_model(model, 'model.pkl')
        return render_template('result.html', accuracy=accuracy)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
'''

with open('litellm_ml_app/app/routes.py', 'w') as f:
    f.write(routes_code)

index_html = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>LiteLLM ML App</title>
</head>
<body>
    <h1>Upload Dataset to Train Model</h1>
    <form action="/train" method="post" enctype="multipart/form-data">
        <input type="file" name="datafile">
        <input type="submit" value="Train Model">
    </form>
</body>
</html>
'''

result_html = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Training Result</title>
</head>
<body>
    <h1>Model Training Complete</h1>
    <p>Model Accuracy: {{ accuracy }}</p>
    <a href="/">Go back</a>
</body>
</html>
'''

with open('litellm_ml_app/app/templates/index.html', 'w') as f:
    f.write(index_html)

with open('litellm_ml_app/app/templates/result.html', 'w') as f:
    f.write(result_html)

print("User interface created successfully.")


User interface created successfully.


In [6]:
from pyngrok import ngrok
import threading
import os

# Function to run Flask app
def run_flask():
    os.system('flask run --no-reload')

# Authenticate with your ngrok authtoken
!ngrok config add-authtoken 2e5YpXo7VVuueVwjFmmiM9706tv_6MfqxYhAH2EpUNuAWrLVR

# Set the FLASK_APP environment variable
os.environ['FLASK_APP'] = 'litellm_ml_app/app/routes.py'

# Start Flask app in a separate thread
thread = threading.Thread(target=run_flask)
thread.start()

# Wait for Flask to start and then connect ngrok
import time
time.sleep(2)  # Give Flask time to start

# Connect ngrok and print the public URL
public_url = ngrok.connect(port='5000')
print(f" * Running on {public_url}")


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


Traceback (most recent call last):
  File "/usr/local/bin/flask", line 5, in <module>
    from flask.cli import main
  File "/usr/local/lib/python3.10/dist-packages/flask/__init__.py", line 6, in <module>
    from .app import Flask as Flask
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 28, in <module>
    from . import cli
  File "/usr/local/lib/python3.10/dist-packages/flask/cli.py", line 18, in <module>
    from click.core import ParameterSource
ImportError: cannot import name 'ParameterSource' from 'click.core' (/usr/local/lib/python3.10/dist-packages/click/core.py)
t=2024-08-15T23:01:24+0300 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configura

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/tunnels/agents\r\n\r\nERR_NGROK_108\r\n.